In [1]:
import numpy as np
import pandas as pd
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares

/Users/tgayanvitharanagamage/anaconda3/envs/rec-sys-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Experiment with the given example dataset

In [7]:
artists, users, artist_user_plays = get_lastfm()

In [11]:
type(artist_user_plays)

scipy.sparse._csr.csr_matrix

In [3]:
# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
# https://benfred.github.io/implicit/tutorial_lastfm.html#:~:text=The%20first%20step,classic%20information%20retrieval%3A
# https://en.wikipedia.org/wiki/Okapi_BM25
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

In [4]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(user_plays)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [21:36<00:00, 86.44s/it]


In [5]:
# Get recommendations for the a single user
userid = 12345
ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, user_plays[userid].indices)})

,artist,score,already_liked
0,the coffinshakers,1.028942,True
1,mortiis,1.020929,True
2,storm,0.975037,False
3,d-a-d,0.972782,True
4,in slaughter natives,0.963969,False
5,von thronstahl,0.961098,True
6,puissance,0.960045,True
7,triarii,0.952944,True
8,kreuzweg ost,0.950946,True
9,arditi,0.948541,True


In [6]:
# get related items for the beatles (itemid = 25512)
ids, scores= model.similar_items(252512)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

,artist,score
0,the beatles,1.000000
1,the beach boys,0.993353
2,the rolling stones,0.993140
3,john lennon,0.992786
4,bob dylan,0.992122
5,the who,0.992069
6,david bowie,0.991860
7,simon & garfunkel,0.991555
8,led zeppelin,0.990879
9,pink floyd,0.990141


In [ ]:
# Make recommendations for the first 1000 users in the dataset
userids = np.arange(1000)
ids, scores = model.recommend(userids, user_plays[userids])
ids, ids.shape

## Experiment with amazon beauty dataset

In [17]:
import numpy as np
import pandas as pd
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from utils import pandas_df_to_csr

In [3]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [4]:
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [5]:
amazon_beauty_csr_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

In [6]:
amazon_beauty_csr_bm25 = amazon_beauty_csr_bm25.tocsr()

In [7]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(amazon_beauty_csr_bm25)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [06:22<00:00, 25.51s/it]


In [15]:
# Get recommendations for the a single user
userid = 725046
ids, scores = model.recommend(userid, amazon_beauty_csr_bm25[userid], N=10, filter_already_liked_items=False)

In [36]:
pd.DataFrame({"ProductId": item_map.loc[ids]["ProductId"], "score": scores, "already_purchased": np.in1d(ids, amazon_beauty_csr_bm25[userid].indices)})

,ProductId,score,already_purchased
ItemIndex,,,
81854,B002OVV7F0,0.294761,True
65766,B001MA0QY2,0.231470,False
141189,B005FYJB92,0.201351,False
88712,B0030O3VRW,0.196237,False
82735,B002QI1F8A,0.172364,False
63375,B001JKTTVQ,0.158884,False
18898,B000GHWSG6,0.155218,False
82527,B002QANC2A,0.153267,False
71340,B001V3TVEQ,0.148638,False


In [13]:
item_map.loc[ids]["ProductId"]

ItemIndex
114276    B0049WJA9C
65766     B001MA0QY2
181492    B008O4YM4Y
10253     B0009PVV40
103349    B003S516XO
174368    B0085WHBHU
75617     B002B9DWBC
89252     B00325D0WK
73976     B0027A7CLG
181069    B008MP481M
Name: ProductId, dtype: object

## End of Experiments

In [42]:
import numpy as np
import pandas as pd
import gc
from implicit.datasets.lastfm import get_lastfm
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from implicit import evaluation
from utils import pandas_df_to_csr

## Pre-processing

In [21]:
amazon_beauty_df = pd.read_csv("ratings_Beauty.csv")

In [22]:
# Convert pandas df to CSR format
# user_map and item_map contains index in csr format to original id mappings for users and items repectively
user_map, item_map, amazon_beauty_csr = pandas_df_to_csr(amazon_beauty_df)

In [23]:
# weight the matrix, both to reduce impact of users that have puchased the same item thousands of times
# and to reduce the weight given to popular items
# Output is a COO matrix
amazon_beauty_coo_bm25 = bm25_weight(amazon_beauty_csr, K1=100, B=0.8)

### Train-test split

In [29]:
train_coo, test_coo = evaluation.train_test_split(amazon_beauty_coo_bm25, train_percentage=0.8, random_state=55)
print(f"Train size: {train.size} \n Test size: {test.size}")

Train size: 1618938 
 Test size: 404132


In [30]:
#Convert coo to csr
train_csr = train_coo.tocsr()
test_csr = test_coo.tocsr()

## Training

In [31]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(train_csr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [05:20<00:00, 21.36s/it]


## Evaluation

In [35]:
ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
ranking_metrics_at_10

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:57<00:00, 5780.66it/s]


{'precision': 0.0189013408792561,
 'map': 0.00874428765414266,
 'ndcg': 0.011266783263312333,
 'auc': 0.5084291897637904}

In [38]:
ranking_metrics_at_10['precision']

0.0189013408792561

## Hyper-parameter Tuning

In [37]:
# Hyper-parameters lists
latent_factors = [32, 64, 128, 256]
regularization = [0.001, 0.005, 0.01, 0.05, 0.1]
alpha = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]

In [43]:
results = []
for f in latent_factors:
    for r in regularization:
        for a in alpha:
            print("Training model with below parameter values --------")
            print(f"latent_factors: {f}, regularization: {r}, alpha: {a}")
            model =  AlternatingLeastSquares(factors=f, regularization=r, alpha=a)
            model.fit(train_csr)
            ranking_metrics_at_10 = evaluation.ranking_metrics_at_k(model, train_csr, test_csr, K=10, show_progress=True, num_threads=0)
            print("Evaluation results: \n", ranking_metrics_at_10)
            results.append(
                (f,r,a,ranking_metrics_at_10['precision'],ranking_metrics_at_10['map'],ranking_metrics_at_10['ndcg'],ranking_metrics_at_10['auc'])
            )
            # Garbage handling
            del model
            gc.collect()

Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 0.5


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:55<00:00, 5932.94it/s]


Evaluation results: 
 {'precision': 0.01006593548933499, 'map': 0.004695287763652912, 'ndcg': 0.006078225427714033, 'auc': 0.504619818152965}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 1.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:54<00:00, 6070.11it/s]


Evaluation results: 
 {'precision': 0.011246646556383936, 'map': 0.005015572210796884, 'ndcg': 0.006602003745638734, 'auc': 0.505113884888769}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 1.5


...

Evaluation results: 
 {'precision': 0.011891803025509835, 'map': 0.005298613546232081, 'ndcg': 0.006975073372547329, 'auc': 0.5054218950506194}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 2.0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [01:11<00:00, 4627.74it/s]


Evaluation results: 
 {'precision': 0.012646561365838591, 'map': 0.005482889526293714, 'ndcg': 0.007273821736302031, 'auc': 0.5057046348963756}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 2.5


 73%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                       | 11/15 [41:01<14:55, 223.77s/it]


KeyboardInterrupt: 

Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 0.5
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [5:11:20<00:00, 1245.35s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:55<00:00, 5932.94it/s]
Evaluation results: 
 {'precision': 0.01006593548933499, 'map': 0.004695287763652912, 'ndcg': 0.006078225427714033, 'auc': 0.504619818152965}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 1.0
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [12:14:54<00:00, 2939.63s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:54<00:00, 6070.11it/s]
Evaluation results: 
 {'precision': 0.011246646556383936, 'map': 0.005015572210796884, 'ndcg': 0.006602003745638734, 'auc': 0.505113884888769}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 1.5
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [1:11:02<00:00, 284.18s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [00:55<00:00, 6008.76it/s]
Evaluation results: 
 {'precision': 0.011891803025509835, 'map': 0.005298613546232081, 'ndcg': 0.006975073372547329, 'auc': 0.5054218950506194}
Training model with below parameter values --------
latent_factors: 32, regularization: 0.001, alpha: 2.0
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [47:07<00:00, 188.47s/it]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 330846/330846 [01:11<00:00, 4627.74it/s]
Evaluation results: 
 {'precision': 0.012646561365838591, 'map': 0.005482889526293714, 'ndcg': 0.007273821736302031, 'auc': 0.5057046348963756}

In [ ]:
results_df = pd.DataFrame(results, columns=['latent_factors', 'regularization', 'alpha', 'precision', 'map', 'ndcg', 'auc'])
results_df